In [ ]:
import os
import pandas as pd
import numpy as np
import datetime

In [ ]:
rebalData =   # 전체 리밸런싱 스케줄 불러오기
priceData =   # 전체 가격 데이터 불러오기

In [ ]:
# 가격데이터가 이미 업로드되어 있다는 전제하에 실행가능한 함수

def get_stock_price(stockCodes, date_start, date_end):
    
    
    return stock_price


def get_basket_history(stockCodes, numStock, date_start, date_end):
    basketPriceData =  get_stock_price(stockCodes, date_start, date_end).fillna(0)
    dates = basketPriceData.index
    basket_history = basketPriceData[stockCodes].values.dot(numStock)    
    basket_history = pd.DataFrame(priceHistory, index = dates)
    return basket_history


In [ ]:
dollar_inv = 100
dollar_ongoing = 100
tradeCose = 0.008
basket_history = {}
rebal_date = list(set(rebalData.date))
rebal_date.sort()
tradeCost_history = {}
turnover_history = {}
weight_history = {}
inv_money_list = [dollar_inv]   
inv_money_date = [rebal_date[0]]

In [ ]:
for i in range(len(rebal_date)-1):
   
    stock_list_i = rebalData[rebalData['date'] == rebal_date[i]].code  # 기간별 종목 리스트 
    w_ = rebalData[rebalData.date == rebal_date[i]].weight 
    
    weightData = pd.concat([stock_list_i, w_], axis = 1).set_index('code')        

    if i == 0:
        weightData['money'] = weightData['weight'] * dollar_inv  # 1기인 경우 초기 투자금으로 시작
    else:
        weightData['money'] = weightData['weight'] * dollar_ongoing # 1기 이후는 투자금의 매 기간별 마지막 시점에서의 금액 재투자            

    stock_price_ = get_stock_price(stock_list_i, rebal_date[i], rebal_date[i]).transpose()  # 해당 일자의 바스켓의 주가 추출     

    weightData = pd.merge(weightData, stock_price_, how = 'inner', left_on= weightData.index, 
                          right_on = stock_price_.index).set_index('key_0')

    weightData.columns = ['weight', 'money', 'price']
    weightData['n_stocks'] = weightData['money'] / weightData['price']

    basket_price = basket_history(weightData.index.values, weightData['n_stocks'].values, rebal_date[i], rebal_date[i+1])     

    dollar_ongoing = basket_price.iloc[-1,:].values[0]  # 투자금의 매 기간별 마지막 시점에서의 포트의 가치(금액)

    # 투자기간 마지막 시점 (다음리밸 시점과 동일) 에서의 가격
    last_price = get_stock_price(weightData.index.values, rebal_date[i+1], rebal_date[i+1]).transpose()  

    weightData = pd.merge(weightData, last_price, how = 'inner', 
                          left_on = weightData.index, right_on = last_price.index).set_index('key_0')

    weightData.columns = ['weight', 'money', 'price', 'n_stocks', 'last_price']

    weightData['last_weight'] = (weightData['last_price'] * weightData['n_stocks']) / (weightData['last_price'] * weightData['n_stocks']).sum()  # 투자기간의 마지막 시점에서의 비중         
    weightData['new_weight'] = rebalData[rebalData['date'] == rebal_date[i+1]].set_index('code')['weight']  # 새로운 비중            
    tradingCost = np.abs(weightData['new_weight'] - weightData['last_weight']).sum() * tradeCost * dollar_ongoing

    dollar_ongoing -= tradingCost     

    inv_money_list += list(basket_price.values.squeeze()[1:-1])
    inv_money_list += [dollar_ongoing]

    inv_money_date += list(basket_price.index[1:])  
    
    basket_history[rebalDate] = basket_price
    tradeCost_history[''.join([x for x in str(rebal_date[i+1])[:10] if x != '-'])] = tradingCost
    turnover_history[''.join([x for x in str(rebal_date[i+1])[:10] if x != '-'])] = tradingCost / dollar_ongoing
    weight_history[''.join([x for x in str(rebal_date[i+1])[:10] if x != '-'])] = weightData


In [ ]:
inv_money_history = pd.DataFrame(inv_money_list, index = inv_money_date) 